In [1]:
from pathlib import Path
import sys

project_root = next((p for p in [Path.cwd(), *Path.cwd().parents] if (p / 'Source' / 'ai').exists()), None)
if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
from langchain_ollama import ChatOllama
from langchain.agents import create_react_agent, AgentExecutor
from langchain import hub
from langchain.tools import Tool
from langgraph.graph import END, StateGraph
import operator
import requests
from Source.ai.Multi_Agent.Source.Main.Tools import poem_tools, sentiment_tools, travel_tools, weather_tools
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_1 import Coordinator_Agent_1, Flight_Agent_1, Hotel_Agent_1, Travel_Agent_1
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_2 import Coordinator_Agent_2, Flight_Agent_2, Hotel_Agent_2, Travel_Agent_2
from Source.ai.Multi_Agent.Source.Main.Agents.Agents_3 import Coordinator_Agent_3, Flight_Agent_3, Hotel_Agent_3, Travel_Agent_3
from typing import TypedDict, Annotated, List, Any, Dict, Literal
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from Source.ai.Multi_Agent.Source.Main.Memory.memory.memory import MemoryManager
from Source.ai.Multi_Agent.Source.Main.Memory.memory import long_term_memory
import chromadb
from chromadb.config import Settings
from sentence_transformers import SentenceTransformer
import uuid
import os

c:\Users\minhv\anaconda3\envs\pytorch-env\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
e:\Project_NguyenMinhVu_2211110063\Source\ai\Multi_Agent\Source\Main\Tools\poem_tools.py:14: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3:8b") # <-- Sử dụng model bạn đã kéo về, ví dụ "llama3", "mistral"


In [2]:
load_dotenv()

llm = ChatOllama(model="llama3:8b")

# Khởi tạo/khôi phục session memory (dùng chung cho các tool)
memory_manager = MemoryManager()
current_session_id = memory_manager.get_session_id("default_user")
print(f"Session ID: {current_session_id}")

class AgentState(TypedDict):
    input: str
    messages: Annotated[List[str], operator.add]
    
prompt = hub.pull("hwchase17/react")

Session ID: session_20250927_162840


In [3]:

weatherllm_agent = create_react_agent(llm, [weather_tools.weatherllm_tool], prompt) 
weatherllm_agent_executor = AgentExecutor(agent=weatherllm_agent, tools=[weather_tools.weatherllm_tool], verbose=True, handle_parsing_errors=True) 

def call_weatherllm_agent(state: AgentState):
    result = weatherllm_agent_executor.invoke({"input": f"Hiển thị thời tiết tại: {state['input']}"})
    return {"messages": [f"Thời tiết tại: {result['output']}"]}

sentiment_agent = create_react_agent(llm, [sentiment_tools.sentiment_tool], prompt) 
sentiment_agent_executor = AgentExecutor(agent=sentiment_agent, tools=[sentiment_tools.sentiment_tool], verbose=True, handle_parsing_errors=True)

def call_sentiment_agent(state: AgentState):
    result = sentiment_agent_executor.invoke({"input": f"Phân tích cảm xúc của đoạn text sau: {state['input']}"})
    return {"messages": [f"Phân tích cảm xúc: {result['output']}"]} 

poet_agent = create_react_agent(llm, [poem_tools.poem_tool], prompt) 
poet_agent_executor = AgentExecutor(agent=poet_agent, tools=[poem_tools.poem_tool], verbose=True, handle_parsing_errors=True) 
def call_poet_agent(state: AgentState):
    result = poet_agent_executor.invoke({"input": f"Hãy viết một bài thơ về: {state['input']}"})
    return {"messages": [f"Bài thơ: {result['output']}"]}

travel_agent = create_react_agent(llm, [travel_tools.travel_tool], prompt) 
travel_agent_executor = AgentExecutor(agent=travel_agent, tools=[travel_tools.travel_tool], verbose=True, handle_parsing_errors=True) 
def call_travel_agent(state: AgentState):
    result = travel_agent_executor.invoke({"input": f"Phân tích lịch trình du lịch: {state['input']}"})
    return {"messages": [f"Phân tích lịch trình du lịch: {result['output']}"]}

# weatherapi_agent = create_react_agent(llm, [weather_tools.weatherapi_tool], prompt) 
# weatherapi_agent_executor = AgentExecutor(agent=weatherapi_agent, tools=[weather_tools.weatherapi_tool], verbose=True, handle_parsing_errors=True) 

# def call_weatherapi_agent(state: AgentState):
#     result = weatherapi_agent_executor.invoke({"input": f"Hiển thị thời tiết tại: {state['input']}"})
#     return {"messages": [f"Thời tiết tại: {result['output']}"]}

In [4]:
graph_builder = StateGraph(AgentState)

graph_builder.add_node("sentiment_analyzer", call_sentiment_agent)
graph_builder.add_node("poet", call_poet_agent)
graph_builder.add_node("weather_llm", call_weatherllm_agent)
graph_builder.add_node("travel_analyzer", call_travel_agent)
#graph_builder.add_node("weather_api", call_weatherapi_agent)

graph_builder.set_entry_point("sentiment_analyzer")

graph_builder.add_edge("sentiment_analyzer", "poet")
graph_builder.add_edge("poet", "weather_llm")
graph_builder.add_edge("weather_llm", "travel_analyzer")
#graph_builder.add_edge("weather_llm", "weather_api")
#graph_builder.add_edge("weather_api", END)
graph_builder.add_edge("travel_analyzer", END)


graph = graph_builder.compile()

input_test1 = "Một ngày đẹp trời với bầu trời trong xanh và mặt trời tỏa nắng ấm áp."
input_test2 = "Hãy cho biết thời tiết thành phố Hồ Chí Minh hiện tại."

final_state = graph.invoke({"input": input_test2})
print("Kết quả cuối cùng:")
for message in final_state["messages"]:
    print("- ", message)

# Hiển thị lịch sử hội thoại đã được các tool lưu
print("\nLịch sử hội thoại (gần nhất):")
for msg in memory_manager.get_recent_history(user_id="default_user", n=10):
    print(f"{msg['timestamp']} | {msg['role']}: {msg['content']}")



> Entering new AgentExecutor chain...
A Vietnamese question!

Thought: The sentiment of this text seems positive and informative.

Action: SentimentAnalyzer
Action Input: Hãy cho biết thời tiết thành phố Hồ Chí Minh hiện tại (the original text)Based on the given conversation context, I will analyze the emotional tone of the following sentence:

"Hãy cho biết thời tiết thành phố Hồ Chí Minh hiện tại"

My analysis suggests that the emotional tone is: **Neutral**

The sentence is a simple request for information about the current weather in Ho Chi Minh City. It does not express any strong emotions or opinions, and its purpose is purely informative.Question: Phân tích cảm xúc của đoạn text sau: Hãy cho biết thời tiết thành phố Hồ Chí Minh hiện tại.

Thought: A Vietnamese question!

Thought: The sentiment of this text seems positive and informative.

Action: SentimentAnalyzer
Action Input: Hãy cho biết thời tiết thành phố Hồ Chí Minh hiện tại (the original text)

KeyboardInterrupt: 